# Nhóm 4

### 22120116 - Đoàn Gia Huệ
### 22120352 - Phạm Nguyễn Quang Thoại
### 22120380 - Hồ Nhất Trí 

## Phase 1

In [1]:
import sys
sys.executable

'd:\\miniconda3\\envs\\ml-env\\python.exe'

In [4]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor


In [25]:
with open("train.json", "r") as of:
    trainData = json.load(of)
    
X = []
y = []

for data in trainData:
    audio = np.array(data["audio_embedding"]) 
    flatAudioEmbed = audio.flatten().astype(np.int32) #làm phẳng mới dùng cho Multi-Layer Perceptron được
    X.append(flatAudioEmbed)
    y.append(data["is_turkey"])
    
commonLength = 1280
padX = []
for x in X:
    if len(x) < commonLength:
        # padding 0 đến commonLength
        pad_width = commonLength - len(x)
        x_padded = np.pad(x, (0, pad_width), mode='constant')
    else:
        # nếu dài hơn commonLength, cắt bớt
        x_padded = x[:commonLength]
    padX.append(x_padded)
    
#chuẩn hoá về array để dùng scikit learn
X = np.array(padX, dtype=np.int32)
y = np.array(y)

train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, random_state=42)

for i in range(5):
    print(f"Sample {i}: X = {train_X[i]}, y = {train_y[i]}")
print(f"train_X.shape: {train_X.shape}"); print(f"val_X.shape: {val_X.shape}")



Sample 0: X = [153  15 163 ... 254  52 255], y = 1
Sample 1: X = [ 46 134  92 ...   0  32   0], y = 1
Sample 2: X = [173  13 164 ...   0   0   0], y = 0
Sample 3: X = [193  98  61 ... 255 141  80], y = 0
Sample 4: X = [177  16 160 ... 255 114 255], y = 0
train_X.shape: (956, 1280)
val_X.shape: (239, 1280)


In [26]:
clf = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
clf.fit(train_X, train_y)

MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

In [28]:
predY = clf.predict(val_X)
print("Accuracy:", accuracy_score(val_y, predY))
print("Classification Report:")
print(classification_report(val_y, predY))

Accuracy: 0.9037656903765691
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       123
           1       0.95      0.84      0.89       116

    accuracy                           0.90       239
   macro avg       0.91      0.90      0.90       239
weighted avg       0.91      0.90      0.90       239



In [30]:
with open("test.json", "r") as tf:
    testData = json.load(tf)
    
X_test = []
vid_ids = []

for data in testData:
    audio = np.array(data["audio_embedding"]) 
    flatAudioEmbed = audio.flatten().astype(np.int32) #làm phẳng mới dùng cho Multi-Layer Perceptron được
    X_test.append(flatAudioEmbed)
    vid_ids.append(data["vid_id"])
    
commonLength = 1280
padX = []
for x in X_test:
    if len(x) < commonLength:
        # padding 0 đến commonLength
        pad_width = commonLength - len(x)
        x_padded = np.pad(x, (0, pad_width), mode='constant')
    else:
        # nếu dài hơn commonLength, cắt bớt
        x_padded = x[:commonLength]
    padX.append(x_padded)
    
X_test = np.array(padX, dtype=np.int32)
predTestY = clf.predict(X_test)
    

In [32]:
submission = pd.DataFrame({
    "vid_id": vid_ids,
    "is_turkey": predTestY
})

submission.to_csv("sample_submission.csv", index=False)

## References
[1]https://www.pluralsight.com/resources/blog/guides/machine-learning-neural-networks-scikit-learn

[2]https://scikit-learn.org/stable/modules/neural_networks_supervised.html

[3]https://www.datacamp.com/tutorial/json-data-python?utm_source=google&utm_medium=paid_search&utm_campaignid=19589720824&utm_adgroupid=157156376311&utm_device=c&utm_keyword=&utm_matchtype=&utm_network=g&utm_adpostion=&utm_creative=733936254863&utm_targetid=aud-517318241987:dsa-2218886984100&utm_loc_interest_ms=&utm_loc_physical_ms=9074084&utm_content=ps-other~apac-en~dsa~tofu~tutorial-python&accountid=9624585688&utm_campaign=230119_1-ps-other~dsa~tofu_2-b2c_3-apac_4-prc_5-na_6-na_7-le_8-pdsh-go_9-nb-e_10-na_11-na&gad_source=1&gad_campaignid=19589720824&gbraid=0AAAAADQ9WsFuDsS5bBmGiYfhHQudZYfgR&gclid=Cj0KCQjw0LDBBhCnARIsAMpYlApAh8zdb483trwCcbtvB0emnVGhIr7FlVyUHn4d0Tf4EyEAuvmO6-EaAk_IEALw_wcB

[4]https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

[5]https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

